<a href="https://colab.research.google.com/github/biozid-0208/daily-random-programming-commit/blob/main/siamese_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import random
import numpy as np
import tensorflow as tf



In [29]:
def prepareData(inputs: np.ndarray, labels: np.ndarray):
      classesNumbers = 10
      digitalIdx = [np.where(labels == i)[0] for i in range(classesNumbers)]
      pairs = list()
      labels = list()
      n = min([len(digitalIdx[d]) for d in range(classesNumbers)]) - 1
      for d in range(classesNumbers):
        for i in range(n):
            z1, z2 = digitalIdx[d][i], digitalIdx[d][i + 1]
            pairs += [[inputs[z1], inputs[z2]]]
            inc = random.randrange(1, classesNumbers)
            dn = (d + inc) % classesNumbers
            z1, z2 = digitalIdx[d][i], digitalIdx[dn][i]
            pairs += [[inputs[z1], inputs[z2]]]
            labels += [1, 0]    
      return np.array(pairs), np.array(labels, dtype=np.float32)

In [30]:
def createTemplate():
      return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.15),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.15),
        tf.keras.layers.Dense(64, activation='relu'),    
        ])

In [31]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_train /= 255
x_test /= 255
input_shape = x_train.shape[1:]

In [32]:
train_pairs, tr_labels = prepareData(x_train, y_train)
test_pairs, test_labels = prepareData(x_test, y_test)

In [33]:
base_network = createTemplate()

In [34]:
# Create first half of the siamese system
input_a = tf.keras.layers.Input(shape=input_shape)
# Note how we reuse the base_network in both halfs
enconder1 = base_network(input_a)
# Create the second half of the siamese system
input_b = tf.keras.layers.Input(shape=input_shape)
enconder2 = base_network(input_b)

In [35]:
distance = tf.keras.layers.Lambda(    
    lambda embeddings: tf.keras.backend.abs(embeddings[0] - embeddings[1])) \
    ([enconder1, enconder2])

In [36]:
measureOfSimilarity = tf.keras.layers.Dense(1, activation='sigmoid') (distance)

In [37]:
# Build the model
model = tf.keras.models.Model([input_a, input_b], measureOfSimilarity)
# Train
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])

model.fit([train_pairs[:, 0], train_pairs[:, 1]], tr_labels,          
          batch_size=128,epochs=10,validation_data=([test_pairs[:, 0], test_pairs[:, 1]], test_labels))

Epoch 1/10
847/847 [==============================] - 8s 9ms/step - loss: 0.3450 - accuracy: 0.8496 - val_loss: 0.2606 - val_accuracy: 0.9181
Epoch 2/10
847/847 [==============================] - 7s 8ms/step - loss: 0.1799 - accuracy: 0.9331 - val_loss: 0.1719 - val_accuracy: 0.9474
Epoch 3/10
847/847 [==============================] - 7s 8ms/step - loss: 0.1253 - accuracy: 0.9549 - val_loss: 0.1462 - val_accuracy: 0.9564
Epoch 4/10
847/847 [==============================] - 7s 8ms/step - loss: 0.0968 - accuracy: 0.9660 - val_loss: 0.1159 - val_accuracy: 0.9649
Epoch 5/10
847/847 [==============================] - 7s 8ms/step - loss: 0.0795 - accuracy: 0.9720 - val_loss: 0.1073 - val_accuracy: 0.9678
Epoch 6/10
847/847 [==============================] - 7s 8ms/step - loss: 0.0681 - accuracy: 0.9766 - val_loss: 0.0938 - val_accuracy: 0.9721
Epoch 7/10
847/847 [==============================] - 7s 8ms/step - loss: 0.0585 - accuracy: 0.9800 - val_loss: 0.0929 - val_accuracy: 0.9727
Epoch 